In [8]:
!pip install PyMuPDF


  Using cached pymupdf-1.26.1-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
Using cached pymupdf-1.26.1-cp39-abi3-win_amd64.whl (18.5 MB)


In [9]:
import fitz
print(fitz.__doc__)

PyMuPDF 1.26.1: Python bindings for the MuPDF 1.26.2 library (rebased implementation).
Python 3.11 running on win32 (64-bit).



In [23]:
import fitz  # PyMuPDF
import pandas as pd

# Maping filename to readable name
pdf_files = {
    r"C:\Users\Bobby\Downloads\invoices\flipkart-bill_compress.pdf": "Flipkart Invoice 1",
    r"C:\Users\Bobby\Downloads\invoices\flipkart-invoice_compress.pdf": "Flipkart Invoice 2",
    r"C:\Users\Bobby\Downloads\invoices\iphone-15-pro-max-and-charger_compress.pdf": "Apple iPhone Invoice",
    r"C:\Users\Bobby\Downloads\invoices\samsung-galaxy_compress.pdf": "Amazon Samsung Invoice"
}

# Extracted results
invoice_data = []

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join([page.get_text() for page in doc])

def find_between(text, start, end):
    try:
        return text.split(start)[1].split(end)[0].strip()
    except:
        return ""

# Iterating PDFs and extract data
for filepath, label in pdf_files.items():
    text = extract_text_from_pdf(filepath)

    data = {
        "Source": label,
        "Invoice Number": "",
        "Invoice Date": "",
        "Order Number": "",
        "Order Date": "",
        "Customer Name": "",
        "Shipping Address": "",
        "Billing Address": "",
        "Product(s)": "",
        "Total Amount": "",
        "Tax Amount": ""
    }

    if "Flipkart" in label:
        data["Invoice Number"] = find_between(text, "Tax Invoice #", "\n")
        data["Order Number"] = find_between(text, "Order ID:", "\n").split()[0]
        data["Order Date"] = find_between(text, "Order Date:", "\n")
        data["Invoice Date"] = find_between(text, "Invoice Date:", "\n")
        data["Customer Name"] = find_between(text, "Shipping Address", "\n").split("\n")[0]
        data["Shipping Address"] = find_between(text, "Shipping Address", "Billing Address").strip()
        data["Billing Address"] = find_between(text, "Billing Address", "Phone:")
        data["Product(s)"] = "\n".join([line.strip() for line in text.split("\n") if "Smartwatch" in line or "Java" in line])
        data["Total Amount"] = find_between(text, "Grand Total", "\n").replace(":", "").strip()

    elif "Apple" in label:
        data["Invoice Number"] = find_between(text, "Invoice Number", "Invoice Date")
        data["Invoice Date"] = find_between(text, "Invoice Date", "Terms").strip()
        data["Order Number"] = find_between(text, "Order Number:", "Order Date").strip()
        data["Order Date"] = find_between(text, "Order Date:", "Sold To").strip()
        data["Customer Name"] = find_between(text, "Sold To:", "Location").split("\n")[0]
        data["Shipping Address"] = find_between(text, "Location:", "Customer No").strip()
        data["Product(s)"] = "\n".join([line for line in text.split("\n") if "IPHONE" in line or "ADAPTER" in line])
        data["Total Amount"] = find_between(text, "Total", "Amount Due").strip()
        data["Tax Amount"] = find_between(text, "Sales Tax", "Total").strip()

    elif "Amazon" in label:
        data["Invoice Number"] = find_between(text, "Invoice Number :", "Order Date").strip()
        data["Order Number"] = find_between(text, "Order Number:", "Invoice Number").strip()
        data["Order Date"] = find_between(text, "Order Date:", "Invoice Details").strip()
        data["Invoice Date"] = find_between(text, "Invoice Date :", "Sl.").strip()
        data["Customer Name"] = find_between(text, "Billing Address :", "Shipping Address").split("\n")[0]
        data["Shipping Address"] = find_between(text, "Shipping Address :", "IN").strip()
        data["Product(s)"] = "\n".join([line for line in text.split("\n") if "Samsung Galaxy" in line])
        data["Total Amount"] = find_between(text, "TOTAL:", "Amount in Words").split()[-1]
        data["Tax Amount"] = find_between(text, "IGST", "₹27").split()[-1]

    invoice_data.append(data)

# Exporting to Excel
df = pd.DataFrame(invoice_data)
df.to_excel(r"C:\Users\Bobby\Downloads\extracted_invoices.xlsx", index=False)

print("Extraction complete. Excel file saved as 'Extracted_Invoices.xlsx'")


Extraction complete. Excel file saved as 'Extracted_Invoices.xlsx'
